## Classic Neural Network for entanglement detection

In this notebook we are going to use fully connected neural network for the task of entanglement detection using observable mean values as features.\
The dataset is imported with pandas and the network is implemented with the torch library.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch as th
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, TensorDataset

from tqdm.auto import trange

### Dataset

We use the dataset of the quantum state: the ```ds_haar_op```.
It's composed by 10 features and 1 label, 0 or 1 if the state is entangled or separable.\
The 10 features represent the means values of some observables (see the dataset folder for an extensive explanation of how the dataset is generated).


Then the dataset is split in train and test part using the hypeparameters define below. 


In [10]:
# Hyperparameters value

N_DATA = 1000
N_TRAIN = 800
N_TEST = N_DATA - N_TRAIN
BATCH_SIZE = 1
EPOCHS = 15
LEARNING_RATE = 0.001

dataset_U = pd.read_csv('../datasets/ds_haar_obs.csv') # import the dataset
dataset_U = dataset_U[:N_DATA] # select the number of data

# Separate features (X) and labels (y)

# Drop the 11th column
X = dataset_U.drop(columns=dataset_U.columns[10])
X = np.array(X)
X = th.tensor(X, dtype=th.float32)

y = dataset_U.iloc[:, 5]  # Assuming the label is in the 11th column (index 10)
y = th.tensor(y.to_numpy(dtype=int)) # convert the labels in torch tensor

X = X[:,0:3] # select only some of the features

#train
obs_train = X[:N_TRAIN]
y_train = y[:N_TRAIN]

train_mapped_dataset = TensorDataset(obs_train,y_train) # create dataset
train_mapped_loader = DataLoader(train_mapped_dataset, shuffle=True, batch_size=BATCH_SIZE) # create dataloader for the training


#TEST
obs_test = X[N_TRAIN:N_DATA]
y_test = y[N_TRAIN:N_DATA]

test_mapped_dataset = TensorDataset(obs_test,y_test) # create your datset
test_mapped_loader = DataLoader(test_mapped_dataset, shuffle=False, batch_size=BATCH_SIZE) # create dataloader for the test


### Model

Define the model as an inherint class from the nn.Module of pytorch.
It's define the constructor, i.e. the architecture of the network with:
* input layer 
* hidden layer 
* output layer

Then is the define a function for the forward part of the training in which is applied an activaction function to each layer and return the ouput of the neural network

In [19]:
# Define the neural network model through a class:

#!!! WARNING: Change in the input layer the number of features

class Net_cluster1(nn.Module):

    # define the model with the constructor of the class
    def __init__(self):
        super(Net_cluster1, self).__init__() 
        self.l1 = nn.Linear(in_features = 3 , out_features = 15) # input layer
        self.l2 = nn.Linear(in_features = 15, out_features = 5) # hidden layer
        self.term = nn.Linear(in_features = 5, out_features = 2) # output layer

    # this function is used for the forward training part
    def forward(self, x: th.Tensor) -> th.Tensor:
        
        x : th.Tensor = x.flatten(start_dim=1)
        x : th.Tensor = F.relu(self.l1(x))
        x : th.Tensor  = F.relu(self.l2(x))
        #x : Tensor = F.relu(self.l3(x))
        logits : th.Tensor = self.term(x)
        out: th.Tensor = F.softmax(input=logits, dim=1)

        return out

In [20]:
def get_batch_accuracy(logit, target):
    """
    Obtain accuracy for one batch of data
    Input:
        - logit(torch.tensor): The predictions from the model 
        - target(torch.tensor): The y true values 
    Return:
        - accuracy(float): The value of the accuracy
    
    """
    corrects = (th.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects / target.size(0)
    return accuracy.item()


def train_loop(model, train_loader, EPOCHS, optimizer, criterion):
    """
    Function for training the model.
    Input:
        - model(torch model): The neural network
        - train_loader(torch DataLoader): The train dataset passed as a torch DataLoader
        - EPOCHS(int): Number of epochs for the training
        - optimizer(torch optimizer): Optimizer for the learning algorithm
        - criterion(torch loss functions): The loss function for the learning algorithm
    Return: Print the loss and train accuracy and train the model
    """

    # do the training loop through the number of epochs
    for epoch in trange(EPOCHS):
        train_running_loss = 0.0
        train_acc = 0.0

        model = model.train()  # Set the model to training mode: relevant for dropout, batchnorm, etc.

        # Actual (batch-wise) training step
        for i, (images, labels) in enumerate(train_loader):
            # Forward pass + (automated) BackProp + Loss computation
            logits = model(images)
            loss = criterion(logits, labels) # calculate the loss

            optimizer.zero_grad()  # Reset the gradients to zero: otherwise they accumulate!
            loss.backward()  # Backpropagation

            # Update model params
            optimizer.step()

            train_running_loss += loss.detach().item()
            train_acc += get_batch_accuracy(logits, labels)

        model.eval()

        print(f"Epoch: {epoch+1} | Loss: {train_running_loss/i} | Train Accuracy: {train_acc/i}")


### Trainig and test part 

Here some important things for the training are defined:
* The model
* The loss function
* The optimizer

Then is used the train_loop function for the training part and after is calculated the test accuracy

In [21]:
model = Net_cluster1() # define the model as an object of the model class
criterion = nn.CrossEntropyLoss()  # Loss function
optimizer = th.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)  # the optimizer

In [22]:

train_loop(model, train_mapped_loader, EPOCHS, optimizer, criterion) # training part
print("")

# calculate the test accuracy
test_acc = 0.0
for i, (dens_matr, labels) in enumerate(test_mapped_loader):
    outputs = model(dens_matr)
    test_acc += get_batch_accuracy(outputs, labels)

print(f"Test Accuracy: {test_acc/i}")

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch: 1 | Loss: 0.4066555391488893 | Train Accuracy: 96.62077596996245
Epoch: 2 | Loss: 0.31459855977375906 | Train Accuracy: 100.12515644555694
Epoch: 3 | Loss: 0.31387713760398656 | Train Accuracy: 100.12515644555694
Epoch: 4 | Loss: 0.31374435479262 | Train Accuracy: 100.12515644555694
Epoch: 5 | Loss: 0.3136979116963803 | Train Accuracy: 100.12515644555694
Epoch: 6 | Loss: 0.313677508370002 | Train Accuracy: 100.12515644555694
Epoch: 7 | Loss: 0.3136673170947312 | Train Accuracy: 100.12515644555694
Epoch: 8 | Loss: 0.3136617357053506 | Train Accuracy: 100.12515644555694
Epoch: 9 | Loss: 0.3136585580840129 | Train Accuracy: 100.12515644555694
Epoch: 10 | Loss: 0.3136566995231619 | Train Accuracy: 100.12515644555694
Epoch: 11 | Loss: 0.3136555687133899 | Train Accuracy: 100.12515644555694
Epoch: 12 | Loss: 0.31365487766504585 | Train Accuracy: 100.12515644555694
Epoch: 13 | Loss: 0.31365445565819294 | Train Accuracy: 100.12515644555694
Epoch: 14 | Loss: 0.3136541910180461 | Train Ac

### Plot
